# Search Engine

In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)
seed = 2021

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = [16, 9]

## Preparing documents

In [2]:
import os

In [4]:
docs = []

DOCS_DIR = 'docs'
for dfile in os.listdir(DOCS_DIR):
    with open(os.path.join(DOCS_DIR, dfile), 'r', encoding='utf-8') as file:
        docs.append(file.read())
len(docs)

50447

## Creating *bag-of-words*

In [3]:
from collections import Counter
import string

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
# # uncomment and execute if the problem occurs below
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# # add another download queries if needed

In [5]:
# code from BIT Python (I am one of the teachers there)
def create_bag_of_words(text, idxs=dict(), update=False):
    text = text.lower()

    # partition into sentences and words
    sentences = sent_tokenize(text)
    words = []
    for sentence in sentences:
        sentence_words = word_tokenize(sentence)
        words.extend(sentence_words)

    # remove stop words
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]

    # remove punctuation
    punctuation = set(string.punctuation)
    punctuation.add("...")
    words = [word for word in words if word not in punctuation]

    # change words to their stems
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words]

    # remove too short stems
    words = [word for word in words if len(word) >= 3]

    if not update:
        vec = np.zeros(len(idxs))
        for word in words:
            idx = idxs.get(word, -1)
            if idx >= 0:
                vec[idxs[word]] += 1
        return vec
    else:
        for word in words:
            if word not in idxs:
                idxs[word] = len(idxs)
        return idxs

In [7]:
%%time
block_size = 500
offset = 0
idxs = dict()
while offset < len(docs):
    create_bag_of_words(' '.join(docs[offset:offset+block_size]), 
                        idxs=idxs, update=True)
    offset += block_size

Wall time: 34min


In [8]:
len(idxs), idxs

(860462,
 {'imagin': 0,
  'imag': 1,
  'audio': 2,
  'file': 3,
  'would': 4,
  'like': 5,
  'transfer': 6,
  'friend': 7,
  'send': 8,
  'raw': 9,
  'format': 10,
  'data': 11,
  'could': 12,
  'time-consum': 13,
  'potenti': 14,
  'ineffici': 15,
  'especi': 16,
  'size': 17,
  'big': 18,
  'convert': 19,
  'origin': 20,
  'bit': 21,
  'compress': 22,
  'sourc': 23,
  'make': 24,
  'much': 25,
  'faster': 26,
  'speed': 27,
  'right': 28,
  'autoencod': 29,
  'save': 30,
  'valuabl': 31,
  'space': 32,
  'instead': 33,
  'bottleneck': 34,
  'slower': 35,
  'uncompress': 36,
  'post': 37,
  'discuss': 38,
  'tensorflow': 39,
  'v2.4.befor': 40,
  'get': 41,
  'technic': 42,
  'detail': 43,
  'let': 44,
  'look': 45,
  'interest': 46,
  'applic': 47,
  'use': 48,
  'remov': 49,
  'nois': 50,
  'denois': 51,
  '.clear': 52,
  'clarifi': 53,
  'imagefil': 54,
  'miss': 55,
  'piec': 56,
  'inpaint': 57,
  '.demo': 58,
  'fill': 59,
  'imagedimension': 60,
  'reduct': 61,
  'cluster': 62,

##### Saving ```idxs``` to JSON

In [7]:
import json

In [10]:
with open('data/idxs.json', 'w') as json_file:
    json.dump(idxs, json_file)

##### Loading ```idxs``` from JSON

In [8]:
with open('data/idxs.json', 'r') as json_file:
    idxs = json.load(json_file)

## *Term-by-document* matrix

In [9]:
from scipy import sparse

In [12]:
%%time
A = sparse.csc_matrix((0, 0), dtype=np.uint32)
for i, doc in enumerate(docs):
    A = sparse.hstack((A, create_bag_of_words(doc, idxs).reshape(-1, 1)))
m, n = A.shape

Wall time: 3h 2min 32s


In [13]:
# sparse.save_npz('data/Araw.npz', A)

In [42]:
A = sparse.load_npz('data/Araw.npz')
m, n = A.shape
m, n

(860462, 50447)

##### Numpy

In [ ]:
# # for numpy
# def idf(A):
#     return A * np.log(n / np.sum(A > 0, axis=1))[:, np.newaxis]

In [ ]:
# A = idf(A)

In [ ]:
# A = A / np.linalg.norm(A, axis=0)

In [ ]:
# A = A.T

##### Scipy

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
transformer = TfidfTransformer()

In [13]:
A = transformer.fit_transform(A.transpose()).transpose()

In [14]:
idfs = transformer.idf_

In [15]:
idfs

array([ 4.62976714,  3.36343026,  4.75204457, ..., 11.1355512 ,
       11.1355512 , 11.1355512 ])

As I have found out, I could have done everything using ```sklearn.feature_extraction.text.TfIdfVectorizer``` :)

Still, it wasn't too late to use at least ```TfidfTransformer``` ;)

In [16]:
# np.save('data/idfs.npy', idfs)

In [ ]:
idfs = np.load('data/idfs.npy')

In [19]:
# sparse.save_npz('data/A.npz', A)

## Search Engine

In [126]:
A = sparse.load_npz('data/A.npz')
m, n = A.shape
m, n

(860462, 50447)

In [24]:
with open('data/docs.json', 'r') as json_file:
    docs = json.load(json_file)

### COSINE similarity

In [127]:
def cosine(q, A):
    return A.transpose().dot(q)

### Input

In [135]:
def process_query(query, A, idxs, idfs, k=20):
    q = create_bag_of_words(query, idxs=idxs)
    # it is not neccessary, because it multiplies all the results by the constant for same
    # entry, so the sorted order is not being changed
    q = q * idfs
    q = q / np.linalg.norm(q)
    print(q[q > 0], np.where(q > 0)[0])
    
    sim = cosine(q, A)

    indices = np.argpartition(sim, sim.shape[0] - k)[-k:]
    return indices[np.argsort(sim[indices])][::-1]

#### Testing out

In [136]:
query = 'tracking algorithm opencv'

In [137]:
indices = process_query(query, A, idxs, idfs)
indices

[0.58830665 0.68879572 0.42362217] [ 196  832 3997]


array([  390,   398,   386,   612,   332,    91,   393,   339,   370,
         389,   202,   568, 39365, 50191,   395,   404,   334, 24703,
         353,   402], dtype=int64)

In [139]:
for i in indices:
    print(docs[i]['name'])

OpenCV 3 adoption rate - PyImageSearch
OpenCV 3.0 released — and the coming changes to the PyImageSearch blog. - PyImageSearch
Checking your OpenCV version using Python - PyImageSearch
Algorithm - Wikipedia
How to install OpenCV 4 on Ubuntu - PyImageSearch
OpenCV Installation on Ubuntu, macOS, Windows and Raspberry Pi | Learn OpenCV
Installing OpenCV on your Raspberry Pi Zero - PyImageSearch
Install OpenCV 4 on macOS - PyImageSearch
Face detection with OpenCV and deep learning - PyImageSearch
Install OpenCV 3 and Python 2.7+ on Ubuntu - PyimageSearch
OpenCV (C++ vs Python) vs MATLAB for Computer Vision | Learn OpenCV
Install OpenCV 4 on Raspberry Pi 4 and Raspbian Buster - PyImageSearch
Search algorithm - Wikipedia
Algorithm (C++) - Wikipedia
How to install OpenCV 3 on Raspbian Jessie - PyImageSearch
Install OpenCV 3.0 and Python 2.7+ on OSX - PyImageSearch
Ubuntu 18.04: How to install OpenCV - PyImageSearch
Exact algorithm - Wikipedia
macOS: Install OpenCV 3 and Python 2.7 - PyImageSe

The perfect results would be tutorials about tracking algorithms posted on Computer Vision blogs, like PyImageSearch or LearnOpenCV. Instead we get lots of tutorials about OpenCV. That's really interesting. I have done this search before having only these 2 blogs as documents (no Wikipedia). And back then, this query gave much better results. 

The problem is in the following:

* 610 documents have word 'OpenCV'
* 1345 documents have word 'algorithm'
* 4910 documents have word 'track'

So the problem is, that the word 'track' is pretty widely used, often having different meaning. And because 'OpenCV' is a name for a library, it is used in less documents, therefore by applying IDF, we get much more weight for 'OpenCV', than for 'tracking'. That's a very good example to see the influence of IDF. Using it without noise reduction via SVD, we can get a problem like this. Still, it performs quite well, taking into considerations, that we've got lots of results about Computer Vision, and even one of the desired articles 'OpenCV Object Tracking'.

In [212]:
query = 'object tracking'

In [213]:
indices = process_query(query, A, idxs, idfs)
indices

[0.68388409 0.72959068] [ 117 3997]


array([  408, 13342, 43276,   403, 18172,   413, 13332, 36675,  6618,
         406,   405,   444,  5734, 11137,   175,   402, 39317,   409,
        2516, 22429], dtype=int64)

In [214]:
for i in indices:
    print(docs[i]['name'])

Object tracking with dlib - PyImageSearch
Object slicing - Wikipedia
Bing's Hollywood - Wikipedia
Tracking multiple objects with OpenCV - PyImageSearch
BBC Sessions (The Who album) - Wikipedia
OpenCV People Counter - PyImageSearch
Cloning (programming) - Wikipedia
Objection (argument) - Wikipedia
Cost object - Wikipedia
OpenCV Track Object Movement - PyImageSearch
Simple object tracking with OpenCV - PyImageSearch
A gentle guide to deep learning object detection - PyImageSearch
Journalistic objectivity - Wikipedia
Object (grammar) - Wikipedia
Object Tracking using OpenCV (C++/Python)
OpenCV Object Tracking - PyImageSearch
Issue tracking system - Wikipedia
Multi-object tracking with dlib - PyImageSearch
Track (rail transport) - Wikipedia
NCAA Division I Men's Indoor Track and Field Championships - Wikipedia


## Low rank approximation

##### Numpy

In [75]:
# def explained_variance(Sig):
#     return np.square(Sig) / np.sum(np.square(Sig))

In [ ]:
# def lowrank(A, k=None, exp_var=0.4):
#     U, D, V = np.linalg.svd(A)
#     if k is None:
#         k = np.searchsorted(np.cumsum(explained_variance(D)), exp_var)
#     return (U[:, :k] * D[:k]) @ V[:k]

##### Scipy

In [141]:
from sklearn.decomposition import TruncatedSVD

In [142]:
def lowrank(A, k=250):
    svd = TruncatedSVD(n_components=k)
    return svd.fit_transform(A), svd.components_

In [155]:
# you need a lot of memory, so there is a chance you need to delete some variables
# filtering out outputs, functions and modules
list(filter(lambda x: not x.startswith('_') 
            and type(eval(x)) != type(cosine)
            and type(eval(x)) != type(plt), dir()))

['A',
 'Counter',
 'In',
 'Out',
 'SnowballStemmer',
 'TfidfTransformer',
 'TruncatedSVD',
 'docs',
 'exit',
 'get_ipython',
 'i',
 'idfs',
 'idxs',
 'indices',
 'indicessvd',
 'json_file',
 'm',
 'n',
 'query',
 'quit',
 'seed',
 'stopwords',
 'transformer']

k = 250

In [180]:
%%time
Alow, svd_components = lowrank(A.transpose())

Wall time: 3min 22s


In [181]:
Alow.shape, svd_components.shape

((50447, 250), (250, 860462))

In [182]:
np.save('data/Alow.npy', Alow)
np.save('data/Alowcomp.npy', svd_components)

k = 150

In [175]:
%%time
Alow150, svd_components150 = lowrank(A.transpose(), k=150)

Wall time: 2min 3s


In [176]:
Alow150.shape, svd_components150.shape

((50447, 150), (150, 860462))

In [177]:
np.save('data/Alow150.npy', Alow150)
np.save('data/Alowcomp150.npy', svd_components150)

k = 50

In [168]:
%%time
Alow50, svd_components50 = lowrank(A.transpose(), k=50)

Wall time: 37.7 s


In [169]:
Alow50.shape, svd_components50.shape

((50447, 50), (50, 860462))

In [170]:
np.save('data/Alow50.npy', Alow50)
np.save('data/Alowcomp50.npy', svd_components50)

Loading low ranked matrices

In [22]:
# Alow = np.load('data/Alow.npy')
# svd_components = np.load('data/Alowcomp.npy')

In [189]:
# Alow150 = np.load('data/Alow150.npy')
# svd_components150 = np.load('data/Alowcomp150.npy')

In [190]:
# Alow50 = np.load('data/Alow50.npy')
# svd_components50 = np.load('data/Alowcomp50.npy')

#### Testing out

In [183]:
def process_query_svd(query, A, svd_comp, idxs, k=20):
    q = create_bag_of_words(query, idxs=idxs)
    # it is not neccessary, because it multiplies all the results by the constant for same
    # entry, so the sorted order is not being changed
    q = q * idfs
    q = q / np.linalg.norm(q)
    print(q[q > 0], np.where(q > 0)[0])
    
    sim = A.dot(svd_comp.dot(q))

    indices = np.argpartition(sim, sim.shape[0] - k)[-k:]
    return indices[np.argsort(sim[indices])][::-1]

k = 250

In [199]:
query = 'tracking algorithm opencv'

In [194]:
indicessvd = process_query_svd(query, Alow, svd_components, idxs)
indicessvd

[0.68388409 0.72959068] [ 117 3997]


array([  408,   409,   355,   413,   350,   444,   403,   406,   402,
         367,  8686, 32618, 16640,   412,   309,   232,   369,   175,
         371, 38032], dtype=int64)

In [195]:
for i in indicessvd:
    print(docs[i]['name'])

Object tracking with dlib - PyImageSearch
Multi-object tracking with dlib - PyImageSearch
Real-time object detection with deep learning and OpenCV - PyImageSearch
OpenCV People Counter - PyImageSearch
Raspberry Pi: Deep learning object detection with OpenCV - PyImageSearch
A gentle guide to deep learning object detection - PyImageSearch
Tracking multiple objects with OpenCV - PyImageSearch
OpenCV Track Object Movement - PyImageSearch
OpenCV Object Tracking - PyImageSearch
Faster video file FPS with cv2.VideoCapture and OpenCV - PyImageSearch
Album - Wikipedia
Album - Wikipedia
Framing (social sciences) - Wikipedia
Ball Tracking with OpenCV - PyImageSearch
OpenCV 'dnn' with NVIDIA GPUs: 1549% faster YOLO, SSD, and Mask R-CNN - PyImageSearch
OpenCV Social Distancing Detector - PyImageSearch
Human Activity Recognition with OpenCV and Deep Learning - PyImageSearch
Object Tracking using OpenCV (C++/Python)
Saving key event video clips with OpenCV - PyImageSearch
Double album - Wikipedia


In [196]:
query = 'object tracking'

In [197]:
indicessvd = process_query_svd(query, Alow, svd_components, idxs)
indicessvd

[0.68388409 0.72959068] [ 117 3997]


array([  408,   409,   355,   413,   350,   444,   403,   406,   402,
         367,  8686, 32618, 16640,   412,   309,   232,   369,   175,
         371, 38032], dtype=int64)

In [198]:
for i in indicessvd:
    print(docs[i]['name'])

Object tracking with dlib - PyImageSearch
Multi-object tracking with dlib - PyImageSearch
Real-time object detection with deep learning and OpenCV - PyImageSearch
OpenCV People Counter - PyImageSearch
Raspberry Pi: Deep learning object detection with OpenCV - PyImageSearch
A gentle guide to deep learning object detection - PyImageSearch
Tracking multiple objects with OpenCV - PyImageSearch
OpenCV Track Object Movement - PyImageSearch
OpenCV Object Tracking - PyImageSearch
Faster video file FPS with cv2.VideoCapture and OpenCV - PyImageSearch
Album - Wikipedia
Album - Wikipedia
Framing (social sciences) - Wikipedia
Ball Tracking with OpenCV - PyImageSearch
OpenCV 'dnn' with NVIDIA GPUs: 1549% faster YOLO, SSD, and Mask R-CNN - PyImageSearch
OpenCV Social Distancing Detector - PyImageSearch
Human Activity Recognition with OpenCV and Deep Learning - PyImageSearch
Object Tracking using OpenCV (C++/Python)
Saving key event video clips with OpenCV - PyImageSearch
Double album - Wikipedia


k = 150

In [200]:
query = 'tracking algorithm opencv'

In [201]:
indicessvd = process_query_svd(query, Alow150, svd_components150, idxs)
indicessvd

[0.58830665 0.68879572 0.42362217] [ 196  832 3997]


array([332, 393, 395, 339, 392, 357, 389, 383, 351, 358, 334, 388, 338,
       390, 353, 356, 568, 404, 582, 360], dtype=int64)

In [202]:
for i in indicessvd:
    print(docs[i]['name'])

How to install OpenCV 4 on Ubuntu - PyImageSearch
Installing OpenCV on your Raspberry Pi Zero - PyImageSearch
How to install OpenCV 3 on Raspbian Jessie - PyImageSearch
Install OpenCV 4 on macOS - PyImageSearch
Installing OpenCV 3.0 for both Python 2.7 and Python 3+ on your Raspberry Pi 2 - PyImageSearch
Ubuntu 16.04: How to install OpenCV - PyImageSearch
Install OpenCV 3 and Python 2.7+ on Ubuntu - PyimageSearch
Install guide: Raspberry Pi 3 + Raspbian Jessie + OpenCV 3 - PyImageSearch
Optimizing OpenCV on the Raspberry Pi - PyImageSearch
Raspbian Stretch: Install OpenCV 3 + Python on your Raspberry Pi - PyImageSearch
Ubuntu 18.04: How to install OpenCV - PyImageSearch
Install OpenCV 3.0 and Python 3 on Ubuntu - PyImageSearch
Install OpenCV 4 on your Raspberry Pi - PyImageSearch
OpenCV 3 adoption rate - PyImageSearch
macOS: Install OpenCV 3 and Python 2.7 - PyImageSearch
macOS: Install OpenCV 3 and Python 3.5 - PyImageSearch
Install OpenCV 4 on Raspberry Pi 4 and Raspbian Buster - PyI

In [203]:
query = 'object tracking'

In [204]:
indicessvd = process_query_svd(query, Alow150, svd_components150, idxs)
indicessvd

[0.68388409 0.72959068] [ 117 3997]


array([  408,   409,   355,   350,   413,   444,   406,  8686, 32618,
         367,   412, 16640,   371,   403, 38032,   369,   581,   290,
         232,   402], dtype=int64)

In [205]:
for i in indicessvd:
    print(docs[i]['name'])

Object tracking with dlib - PyImageSearch
Multi-object tracking with dlib - PyImageSearch
Real-time object detection with deep learning and OpenCV - PyImageSearch
Raspberry Pi: Deep learning object detection with OpenCV - PyImageSearch
OpenCV People Counter - PyImageSearch
A gentle guide to deep learning object detection - PyImageSearch
OpenCV Track Object Movement - PyImageSearch
Album - Wikipedia
Album - Wikipedia
Faster video file FPS with cv2.VideoCapture and OpenCV - PyImageSearch
Ball Tracking with OpenCV - PyImageSearch
Framing (social sciences) - Wikipedia
Saving key event video clips with OpenCV - PyImageSearch
Tracking multiple objects with OpenCV - PyImageSearch
Double album - Wikipedia
Human Activity Recognition with OpenCV and Deep Learning - PyImageSearch
Multiple cameras with the Raspberry Pi and OpenCV - PyImageSearch
Real-time facial landmark detection with OpenCV, Python, and dlib - PyImageSearch
OpenCV Social Distancing Detector - PyImageSearch
OpenCV Object Tracking

k = 50

In [206]:
query = 'tracking algorithm opencv'

In [207]:
indicessvd = process_query_svd(query, Alow50, svd_components50, idxs)
indicessvd

[0.58830665 0.68879572 0.42362217] [ 196  832 3997]


array([332, 393, 395, 339, 357, 392, 383, 351, 389, 334, 338, 358, 388,
       353, 356, 404, 568, 390, 360, 582], dtype=int64)

In [208]:
for i in indicessvd:
    print(docs[i]['name'])

How to install OpenCV 4 on Ubuntu - PyImageSearch
Installing OpenCV on your Raspberry Pi Zero - PyImageSearch
How to install OpenCV 3 on Raspbian Jessie - PyImageSearch
Install OpenCV 4 on macOS - PyImageSearch
Ubuntu 16.04: How to install OpenCV - PyImageSearch
Installing OpenCV 3.0 for both Python 2.7 and Python 3+ on your Raspberry Pi 2 - PyImageSearch
Install guide: Raspberry Pi 3 + Raspbian Jessie + OpenCV 3 - PyImageSearch
Optimizing OpenCV on the Raspberry Pi - PyImageSearch
Install OpenCV 3 and Python 2.7+ on Ubuntu - PyimageSearch
Ubuntu 18.04: How to install OpenCV - PyImageSearch
Install OpenCV 4 on your Raspberry Pi - PyImageSearch
Raspbian Stretch: Install OpenCV 3 + Python on your Raspberry Pi - PyImageSearch
Install OpenCV 3.0 and Python 3 on Ubuntu - PyImageSearch
macOS: Install OpenCV 3 and Python 2.7 - PyImageSearch
macOS: Install OpenCV 3 and Python 3.5 - PyImageSearch
Install OpenCV 3.0 and Python 2.7+ on OSX - PyImageSearch
Install OpenCV 4 on Raspberry Pi 4 and Ra

In [209]:
query = 'object tracking'

In [210]:
indicessvd = process_query_svd(query, Alow50, svd_components50, idxs)
indicessvd

[0.68388409 0.72959068] [ 117 3997]


array([32618,  8686,  1044, 38032, 19401, 18031, 48987,  6179, 15740,
       35158, 24427, 33267, 48955, 25022, 26577, 22918, 20511, 27159,
       27242, 15364], dtype=int64)

In [211]:
for i in indicessvd:
    print(docs[i]['name'])

Album - Wikipedia
Album - Wikipedia
UK Albums Chart - Wikipedia
Double album - Wikipedia
Example (musician) - Wikipedia
Maurice White - Wikipedia
Kill the Lights (Luke Bryan album) - Wikipedia
Arctic Monkeys - Wikipedia
Feeder (band) - Wikipedia
Train station - Wikipedia
Train station - Wikipedia
When Disaster Strikes... - Wikipedia
Sigh No More (Mumford & Sons album) - Wikipedia
Older (album) - Wikipedia
The Bodyguard (soundtrack) - Wikipedia
The Suburbs - Wikipedia
Elephant (album) - Wikipedia
Colt Ford - Wikipedia
Parachutes (Coldplay album) - Wikipedia
Savage Garden (Savage Garden album) - Wikipedia


## Comparing

Because of the RAM restrictions couldn't test big $k$ values for SVD low rank approximation.

Firstly, we can see, that writing a proper query is absolutely essential for any possible configuration. We can see that results for 'tracking algorithm opencv' rarely gives us good results. Mostly it gives us results about OpenCV library, and here we can see the influence of IDF. The comment about it is above. So let's take a look at results for the 'object tracking' query.

We can see, that results for matrix without low rank approximation contain a lot of wikipedia pages, which have no relation to object tracking, but instead those articles contain lots of 'object' and 'track' words. For example, 'Object slicing - Wikipedia' or 'Object (grammar) - Wikipedia'.

For low rank matrix, we do not get such useless results. Most results have some relation to the object tracking. The problem appears when the value of $k$ gets lower. For instance, the results for $k = 250$ are much more specific, than for $k = 150$. $k = 50$ does not have any sense at all. We get utterly useless results having very little relation to the query.

So, in my opinion, results for $k > 200$ are the best. I have tested some higher values before, and most of them have given pretty satisfying results. I haven't had an opportunity to test big $k$ values, so it is difficult to tell the upper limit of $k$. (I have over 800,000 dimensions, so it takes a lot of memory). So, subjectively, the values between 200 and 500 are the best. But it may strongly depend on the documents we have. Because, if there are too many different clusters of them based on topics, then we need a bigger $k$ to distinguish them.